In [1]:
import os
import docx2txt
import PyPDF2
import time
import pandas
import docx
import re
import glob
import google.generativeai as genai
import os
from dotenv import load_dotenv
import os

genai.configure(
    api_key=os.environ['API_KEY'])
model_gemini = genai.GenerativeModel(
    model_name='gemini-pro')

e:\Research work\assessment checker research\my_protobuf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Function to apply gemini-3.5 model to a given text

In [2]:

# Define function to apply gemini-3.5 model to a given text
# def generate_text(work, desc, rubrics):   
#     description = desc
#     rubrics=rubrics
#     text = work
#     instructor = """a language instructor and assess the work of students. I will provide you the assignment description/task given to students. 
#         Then,  I will provide the student's work and the rubrics. 
#         Your task will be to assign points and comments against each rubric first. 
#         Then, Please highlight words, sentences, or phrases where the student made mistakes or violated rubrics.""" 
#     prompt_aspect = f"""Act as {instructor} 
#         Assignment description: 
#         {description};    
#         Rubrics:
#         {rubrics}; 
#         Student Work: 
#         {text}"""
def generate_text(work, desc, rubrics):
    rubrics = rubrics
    text = work
    instructor = """an experienced language instructor and assess the work of students. 
    I will provide the student's essay, Assignment description given to students, and the rubrics. 
    Your task is to assign a score and provide detailed comments for each rubric. 
    Highlight words, sentences, or phrases where the student made mistakes or did not meet the rubric criteria."""

    prompt_aspect = f"""Act as {instructor} 

    Description:
    {desc}

    Rubrics:
    {rubrics}

    Student Work: 
    {text}

    Instructions:
    1. Assign a score for each rubric criterion based on the student's essay.
    2. Provide detailed comments explaining the score for each criterion.
    3. Highlight specific mistakes or areas where the essay did not meet the rubric criteria.
    4. Offer constructive feedback for improvement where applicable."""
    try:
        completion = model_gemini.generate_content(
            prompt_aspect,
            generation_config={
                'temperature': 0.1,
                'max_output_tokens': 800
            }
        )
        score_gemini = completion.text
    except Exception as e:
        score_gemini = str(e) 
    result = score_gemini    
    # result = desc + rubrics+ text
    return result



# Functions to read files

In [3]:
# Define function to read text from a given Word file
def read_word_file(filename):

    # Open the Word file
    doc = docx.Document(filename)

    # Extract the text from the Word file
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)

    # Print the text from the Word file
    return "\n".join(text)

# Define function to read text from a given PDF file
def read_pdf_file(filename):
    with open(filename, 'rb') as f:
        reader = PyPDF2.PdfFileReader(f)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text

In [4]:

# Define function to process all files in a given folder
def process_file(filename):
    print(filename)
    try:
        if filename.endswith('.docx'):
                text = read_word_file(filename)
        elif filename.endswith('.pdf'):
                doc = fitz.open(filename)
                text = ""
                for page in doc:
                        text+=page.get_text()
                        print(text)
        else:
                text = ""
    except Exception as e:
        text = str(e)
    return text    


## Main Folder of Assignment

In [5]:
folder_path = 'OLevels/'

In [6]:
import pandas as pd

df = pd.read_csv(os.path.join(folder_path, "Results_Gemini/Results analysis_Olevels.csv"))
# df.dropna(subset=['human_Grade'], inplace=True)
print(len(df))

print(df.head(5))


21
  Essay ID       Essay type  human1_Score  human2_Score  GPT_Score
0     e001   Report Writing            23           NaN         23
1     e002  Informal letter            25          27.0         23
2     e003   Report Writing            23           NaN         21
3     e004   Report Writing            25           NaN         23
4     e005     Descriptive             15           NaN         18


In [7]:
df = df.loc[df["Essay type"] == "Descriptive "]
df.head()

,Essay ID,Essay type,human1_Score,human2_Score,GPT_Score
4,e005,Descriptive,15,NaN,18
6,e007,Descriptive,18,NaN,14
8,e009,Descriptive,28,NaN,22
9,e010,Descriptive,28,NaN,22
10,e011,Descriptive,28,NaN,22


## Description and Rucrics

In [8]:
desc = read_word_file(os.path.join(folder_path, 'Description and Rubrics/Descriptive Essay_description.docx'))
rubrics = read_word_file(os.path.join(folder_path, 'Description and Rubrics/Descriptive Essay_rubrics.docx'))
# Create a list of unique rubric names
unique_rubrics = ["Score", "Point", "Rubrics"]
# Create a new dictionary for each unique rubric name to store the assessment results
rubric_results = {rubric: [] for rubric in unique_rubrics}
# df.head(5)
keywords = ["Comments", "Mistakes", "Quality of Language" "Appropriateness and Content"]
results = {rubric: [] for rubric in keywords}
feedback= []

In [9]:
rubrics

'\nThe maximum marks for this assignment are 30.\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t30\t26\t22\t18\t14\t10\t6\t2\n\tBAND-8\tBAND-7\tBAND-6\tBAND-5\tBAND-4\tBAND-3\tBAND-2\tBAND-1\n\t"""Highly accurate writing, apart from very occasional slips; highly appropriate to chosen task type.\n Quality of Language\n · Sentence structures varied for particular effects.\n · Verb forms largely correct and appropriate tenses consistently used.\n · Vocabulary wide and precise.\n · Punctuation accurate and helpful.\n · Spelling accurate apart from very occasional slips.\n · Paragraphs have unity, are linked, and show evidence of planning.\n Appropriateness and Content\n · Consistently relevant. Interest aroused and sustained.\n · Tone and register entirely appropriate.\n · Descriptions have well-developed images helping to create complex atmospheres.\n · Arguments are well developed, logical, even complex.\n · Narratives are complex, sophisticated, possibly tense, and may contain devices such as flashba

In [10]:
desc

'DESCRIPTIVE ESSAY TEST FROM GIVEN CHOICES\nDescribe two very different and interesting animals which live in your area. (Remember you can describe the behaviour of the animals and the places where they live as well as their appearance.)\nDescribe two very different locations you know which have lots of trees, plants and flowers. (Remember you can describe any people there and the atmosphere, as well as the trees, plants and flowers.)\nDescribe the most helpful person and the most unhelpful person you know. (Remember you can describe their appearance as well as their character and behaviour.)'

In [11]:
for index, row in df.iterrows():
        print(row['Essay ID'])
        filename = os.path.join("OLevels/Collected essays/Descriptive/", row["Essay ID"])
        print(filename)


e005
OLevels/Collected essays/Descriptive/e005
e007
OLevels/Collected essays/Descriptive/e007
e009
OLevels/Collected essays/Descriptive/e009
e010
OLevels/Collected essays/Descriptive/e010
e011
OLevels/Collected essays/Descriptive/e011
e013
OLevels/Collected essays/Descriptive/e013
e019
OLevels/Collected essays/Descriptive/e019


# Call the process_files function

# Function to process all files in a given folder

In [12]:
jump=1
path = "E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive/"
for i in range(0,  len(df), jump):
    data = df[i:i+jump] 
    # Iterate over the rows in the data
    for index, row in data.iterrows():
        # filename = os.path.join("OLevels/Collected essays/Informal/", row["Essay ID"])
        filename = next(glob.iglob(os.path.join(path, row["Essay ID"]+'.*')), None)
        print(filename)
        # Get the text to assess
        if filename:
            print(filename)
            text_to_assess = process_file(filename)
        else:
            text_to_assess = "Not Submitted the work"
        print(text_to_assess)
            # Apply the OpenAI model to the text to get the assessment result
        assessment_result = generate_text(work= text_to_assess, desc=desc, rubrics=rubrics)
        feedback.append(assessment_result)    
        print(assessment_result, "\n\n\n")
        # Extract the rubric values from the assessment result text
        for rubric in unique_rubrics:
            pattern = re.compile(rf"(?i){rubric}\D+(\d+(\.\d+)?)")
            match = pattern.search(assessment_result)
            print(match)
            if match:
                rubric_results[rubric].append(match.group(1))
            else:
                rubric_results[rubric].append("")
        for i in range(len(keywords)):
            keyword = keywords[i]
            pattern = re.compile(rf"(?i)\b{keyword}\b(.+?)(?=\b{keywords[i+1]}\b|$)" if i < len(keywords) - 1 else rf"(?i)\b{keyword}\b(.+?$)", re.DOTALL)
            match = pattern.search(assessment_result)
            if match:
                results[keyword].append(match.group(1).strip())
            else:
                results[keyword].append("")   
        time.sleep(20) 
    for rubric in unique_rubrics:
        data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
    for keyword in keywords:
        data[keyword] = results[keyword]   
    data['OverallResponse'] = feedback
    
    # Save the data with the rubric columns to a new CSV file
    data.to_csv(os.path.join(folder_path, "Results_Gemini/Results analysis_Olevels_Gemini.csv"), index=False, header=False, mode='a')

    rubric_results = {rubric: [] for rubric in unique_rubrics}
    results = {rubric: [] for rubric in keywords}
    feedback=[]

E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e005.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e005.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e005.docx
Everyone has an imaginary place where he or she wants to hide themselves from real world troubles and worries. Some go to sleep to dream pleasant and release their minds from the stress, others go to mediate exercise and release the tensions and emotions. I do both of them some time to release my stress and when I am feeling   upset and being emotional.
From my childhood I liked to see nature so my favourite place is wildlife which is located far away from people and cities. I like the high mountains having snow peaks, peak corn, meadow and wood. There is pure air which is filled from the rich smell of flowers and wood. There is no one else except me and the wild animal plus birds flying in the dark blue sky. The w

C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e007.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e007.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e007.docx
 TWO DIFFERENT ANIMALS NEAR ME .

My neighbor has a big villa in which he and his whole family live; he has interest in different animals like gorilla, zebra and more. He usually go to visit at zoo at least once at a weak, sometimes he take me with himself at zoo, he has a great  knowledge about animals he owned some animals in his house, he also hired number of persons to take care of them he owned a tiger which looks dangerous, when my friend goes for hunting  tigers glowing ferocious eyes see everything sharply, his sharp claws help him to attack easily and his 4 big and sharp teeth help him tear anything, he jumps very high, he has a tall thick tail, he have skin coloured shiny hair, with black strips verticall

C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e009.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e009.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e009.docx
Test

Q) Describe two very different locations you know which have lots of trees, plants and flowers. (Remember you can describe any people there and the atmosphere, as well as the trees, plants
and flowers.)


The two different locations which i know are, a dark forest and a beautiful garden in spring weather. 
The trees in the forest are malady brown. Grains of poison begrimed the dark and gleamed like witch dust. Trolls haunted the sooty copies, salivation over their prey and smearing the blood over their heavy faces. The decaying air and stifling atmosphere provided the perfect abode for those who worshiped the darkness rather than the light. 

In the dense shadows, spiders clutched their snare strings, their w

...

C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e013.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e013.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e013.docx
Write a descriptive essay on two very different locations where there are lots of trees, flowers and plants. (Remember you can describe the atmosphere and people there). 

Indeed the vastness of the Amazon forest is nerve-wracking. Till the ending spot my sight could reach, there were gigantic sequoias glaring at me as if I owned their debt. I was encircled by antiquated hickory-brown woods and creepy looking spider orchids so as to cut my escape. Clumps of bushes and overhanging limbs aroused a thrilling vibe in my core and the whole environment seemed celestial and fictional Teribethia perhaps because I have never been in such wild nature. 
The hiss of lime-green snakes griping tree trucks compactly captivated my

C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e019.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e019.docx
E:/Research work/assessment checker research/OLevels/Collected essays/Descriptive\e019.docx
DESCRIPTIVE ESSAY TEST    

Describe two very different locations you know which have lots of trees, plants and flowers. (Remember you can describe any people there and the atmosphere, as well as the trees, plants
and flowers.)      
A sports ground,a walking track,an animal house,one in a million,the Central park of Metropolis,also to be called the Heavenly Orchard of Metropolis.More likely to be said,the heartly treasure of Metropolis.

 Thick hollow trunks Of Greenish Alpine trees appearing to be as like as street poles,patient to provide the never-ending luminosity in the form of relaxing mild shelter of shadows. Greenish blanket of soft grass warmly welcoming the tourists with opened gorgeous tangles. 
The cos

C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["gemini_"+rubric+"_Score"] = rubric_results[rubric]
C:\Users\Nimra\AppData\Local\Temp\ipykernel_4144\1484066637.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo